In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import pandas as pd
import bs4
from selenium import webdriver
import re

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [3]:

#opts = webdriver.ChromeOptions()
#opts.headless = True
driver = webdriver.Chrome()

In [8]:
def get_info(film_tag):
    
    hrefMovie = film_tag.find('div',class_ = 'lister-item-image')
    hrefMovie = hrefMovie.find('a')['href']
    href_link = 'https://www.imdb.com/' + hrefMovie
    driver.get(href_link)
    WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div[data-testid='storyline-plot-keywords']")))
    html_text = driver.page_source
    href_soup = BeautifulSoup(html_text)
    
    keywords_tag = href_soup.find_all('div', class_="ipc-chip-list__scroller")
    keywords = []
    if keywords_tag[1]:
        for tag in keywords_tag[1].contents:
            keywords.append(tag.get_text())
        keywords.pop(-1)
    keywords = ', '.join(keywords)
    tagline_tag = href_soup.select('li[data-testid="storyline-taglines"] label')
    tagline = ''
    if tagline_tag:
        tagline = [i.text for i in tagline_tag][0]
        
    metascore_tag = href_soup.find('span', class_ = 'score-meta')
    metascore = ''
    if metascore_tag:
        metascore = metascore_tag.get_text()
    else:
        metascore = np.nan
    
    details = href_soup.select('div[data-testid="title-details-section"] a[class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"]')
    release_date = ''
    countries = []
    language = []
    locations = []
    company = []
    if details:
        for d in details:
            if 'releaseinfo' in d['href']:
                release_date = d.text
            elif 'country_of_origin' in d['href']:
                countries.append(d.text)
            elif 'primary_language' in d['href']:
                language.append(d.text)
            elif 'locations' in d['href']:
                locations.append(d.text)
            elif 'company' in d['href']:
                company.append(d.text)
    countries = ', '.join(countries)
    language = ', '.join(language)
    company = ', '.join(company)
    locations = ', '.join(locations)
    scraped_list = film_tag.find( 'div', class_ = 'lister-item-content')
    
    name_tag = film_tag.find('h3')
    name = name_tag.get_text().split('\n')[2]
    
    muted_text = film_tag.find_all( 'p', class_ = 'text-muted')
    overview = muted_text[1].get_text().strip()
    
    cer = film_tag.find('span', class_ = 'certificate')
    if cer:
        certificate = cer.get_text()
    else:
        certificate = ''
    
    runtime_tag = film_tag.find('span', class_ = 'runtime')
    if runtime_tag:
        runtime = float(runtime_tag.get_text().split(' ')[0])
    else:
        runtime = np.nan
        
    genre_tag = film_tag.find('span', class_ = 'genre')
    if genre_tag:
        genre = genre_tag.get_text().strip().split(', ')
        genre = ', '.join(genre)
    else: 
        genre = ''
    
    imdb_rate_tag = film_tag.find('strong')
    if imdb_rate_tag:
        imdb_rate = float(imdb_rate_tag.get_text())
    else:
        imdb_rate = np.nan
#     metascore_tag = film_tag.find('span', class_ = 'metascore favorable')
#     if metascore_tag :
#         metascore = float(metascore_tag.get_text().strip())
#     else:
#         metascore =np.nan
    star_tag = film_tag.find('p', class_ = "").get_text().strip()
    star_tag = star_tag.split('|')
    if len(star_tag) > 1:
        director_tag =  star_tag[0].split(':\n')
        star_tag = star_tag[1].split(':\n')
        director = director_tag[1].strip().split(', \n')
        director = ', '.join(director)
        stars = star_tag[1].strip().split(', \n')
        stars = ', '.join(stars)
    else:
        temp = star_tag[0].strip().split(':\n')
        if temp[0] == 'Director':
            director = temp[1]
            director = ', '.join(director)
            stars = ''
        else:
            stars = temp[1]
            stars = ', '.join(stars)
            director = ''
        
    vote_tag = film_tag.find('p', class_ = 'sort-num_votes-visible').get_text()
    vote_tag = vote_tag.split('\n|')
    nvote = vote_tag[0].split(':\n')[1].replace(',', '')
    nvote = int(nvote)
    if len(vote_tag) > 1:
        gross = float(vote_tag[1].strip().split(':\n')[1].strip('$M'))
    else:
        gross = np.nan
    
    result = [name, overview,tagline,certificate, runtime, genre,keywords, imdb_rate, metascore, director, stars, nvote, gross,
              release_date, countries,language, locations, company]
    return result

In [ ]:
film_list = []
# search feature film có nhiều lượt vote từ 2010 - 2021, 
url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2010-01-01,2021-12-31&sort=num_votes,desc&count=250"
while len(film_list) < 1000:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    scraped_list = soup.find_all( 'div', class_ = 'lister-item mode-advanced')
    for film in scraped_list:
        film_list.append(get_info(film))
    next_tag = soup.find('a', class_ = 'lister-page-next next-page')
    url = 'https://www.imdb.com/' + next_tag.get('href')
key = ['name', 'overview','tagline','certificate', 'runtime', 'genre','keywords', 'imdb_rate', 'metascore', 'director', 'stars', 'nvote', 'gross',
              'release_date', 'countries','language', 'locations', 'company']
df = pd.DataFrame(film_list,columns = key)

In [ ]:
driver.quit()

In [ ]:
df

In [ ]:
df.to_csv('data.csv', index = False, sep='\t', encoding='utf-8')